In [1]:
import tensorflow as tf
import numpy as np
from trade_sim import Market
import tensorflow.contrib as contrib
import os
from multiprocessing import Process, Queue, Manager
import random
import math
import time

In [12]:
class Agent:
    def __init__(self, numberOfCurrencies, sess, timeFrame=50,  initialPortfolio=10000.0):
        self._s = sess
        self._isTraining = tf.placeholder(tf.bool)
        self.inputT = tf.placeholder(shape=[None, numberOfCurrencies, timeFrame, 3], dtype=tf.float32)
        self.model = tf.layers.conv2d(inputs=self.inputT, filters=10, kernel_size=[1,3], activation=tf.nn.leaky_relu)
        self.model = tf.layers.conv2d(inputs=self.model, filters=10, kernel_size=[1,48], activation=tf.nn.leaky_relu)
        #self.model = tf.layers.conv2d(inputs=self.model, filters=100, kernel_size=[1,22], activation=tf.nn.leaky_relu)

      #  self.conv1 = tf.nn.depthwise_conv2d(self.inputT, [1,3,3,4], [1,1,1,1], 'SAME')
        #self.conv2 = tf.layers.conv2d(inputs=self.conv1, filters=150, kernel_size=[1,8], activation=tf.nn.relu)
        #self.conv3 = tf.layers.conv2d(inputs = self.conv2, filters=200, kernel_size=[1,41], activation=tf.nn.relu)
        #self.conv4 = tf.layers.conv2d(inputs=self.conv3, filters=30, kernel_size=[1,1] , activation=tf.nn.relu) 
#        self.model = tf.layers.dense(self.model, 2000, activation=tf.nn.leaky_relu)
        self.model = tf.layers.conv2d(inputs=self.model, filters=100, kernel_size=[1,1], activation=tf.nn.leaky_relu)
        self.model = tf.layers.dense(self.model, 1000)
        #self.model = tf.layers.dropout(self.model, training = self._isTraining)
        print(self.model.shape)
        self.model = tf.layers.dense(self.model, 1)
        print(self.model.shape)
        self._allocate = tf.nn.softmax(self.model, axis=1)
        
        self.priceChanges = tf.placeholder(shape=[None, numberOfCurrencies, 1], dtype=tf.float32)
        
        #self.loss = -tf.matmul(tf.matrix_transpose(tf.nn.leaky_relu(tf.log(self.priceChanges), alpha=10)),tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))
        self.averageLoss = tf.reduce_mean(tf.matmul(tf.matrix_transpose(self.priceChanges), 
                                             tf.scalar_mul(tf.constant(initialPortfolio), 
                                               tf.reshape(self._allocate, [-1, numberOfCurrencies, 1]))))

        self.loss = tf.exp(tf.reduce_sum(tf.multiply(-tf.log(self.priceChanges), tf.reshape(self._allocate, [tf.shape(self._allocate)[0], numberOfCurrencies, 1]))))
        self.optimizer = tf.train.AdamOptimizer(learning_rate=0.001)
        self._train = self.optimizer.minimize(self.loss)
        
    def act(self, observation):
        return self._s.run(self._allocate, feed_dict={self.inputT: observation})
    
    def train_step(self, obs, prices):
        batch_feed = {self.inputT : obs,
                     self.priceChanges: prices,
                      self._isTraining: True
                     }
        _, lossValue = self._s.run([self._train, self.averageLoss], feed_dict=batch_feed)
        return lossValue
    def test_model(self, obs, prices):
        print(self._s.run([self.averageLoss, self._allocate], feed_dict={self.inputT : obs, self.priceChanges: prices, self._isTraining: False}))
        print(prices)

In [3]:
class DataManager:
    def __init__(self, batch_size=1, batch_count=0, period_size=50, batch_offset=0):
        self.batchPath = os.path.abspath("./Batches")
        self.labelPath = os.path.abspath("./Labels")
        self.threads = 8
        self.batch = batch_count
        self.batch_size = batch_size
        self.period_size = period_size
        self.offset = batch_offset
        
            
    def importData(self, simulator):
        testSim = simulator
        manager = Manager()
        d = manager.list()
        q = Queue()
        jobs = []
        PERIOD_SIZE = self.period_size
        BATCH_SIZE = self.batch_size
        #BATCH_COUNT = int(math.floor(minSize / BATCH_SIZE))
        if self.batch == 0:
            minSize = math.inf
            for pair in simulator.df.values():
                if len(pair.index) < minSize:
                    minSize = len(pair.index)
            print(minSize)
            self.batch = int(math.floor(minSize / self.batch_size))
        BATCH_COUNT = self.batch
        BATCH_OFFSET = self.offset
        dates = testSim.getAllDates()
        index = list(range(BATCH_COUNT))
        feed = []
        

        running = False
        count = 0
        countFile = 0
#         while 1:
#             if count < self.threads and len(index) != 0:
#                 for i in random.sample(index, self.threads-count if len(index) >= self.threads-count else len(index)):
#                     p = Process(target=testSim.processTimePeriod, args=(d, PERIOD_SIZE, dates, BATCH_SIZE * (i + BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE))
#                     jobs.append(p)
#                     p.start()
#                     index.remove(i)
#                 print("Total unstarted:{}".format(len(index)))
#             count = 0
#             for p in jobs:
#                 if not p.is_alive():
#                     p.terminate()
#                     jobs.remove(p)
#                 else:
#                     count += 1
#             while len(d) != 0:
# #                feed.append(q.get())
#                 pair = d.pop(0)
#                 np.save((os.path.join(self.batchPath, "Batch_" + str(countFile))), pair[0])
#                 np.save((os.path.join(self.labelPath, "Label_" + str(countFile))), pair[1])
#                 countFile += 1
#             if count == 0 and len(index) == 0 and countFile == BATCH_COUNT:
#                 break
        prevIndex = [-1 for i in range(len(simulator.currencies))]
        for i in index:
            prevIndex = testSim.processTimePeriod(d, PERIOD_SIZE, dates, BATCH_SIZE * (i+BATCH_OFFSET) + PERIOD_SIZE, BATCH_SIZE, prevIndex)
            pair = d.pop(0)
           # print('END OF CALL ---------------------')
            np.save((os.path.join(self.batchPath, "Batch_" + str(countFile))), pair[0])
            np.save((os.path.join(self.labelPath, "Label_" + str(countFile))), pair[1])
            countFile += 1
        
#         print(len(feed))
#         return feed
    
    def prepareData(self, simulator, reset=False):
        if not os.path.exists(self.batchPath):
            os.makedirs(self.batchPath)
        if not os.path.exists(self.labelPath):
            os.makedirs(self.labelPath)
        if reset:
            for folder in (self.batchPath, self.labelPath):
                for the_file in os.listdir(folder):
                    file_path = os.path.join(folder, the_file)
                    try:
                        if os.path.isfile(file_path):
                            os.unlink(file_path)
                    except:
                        pass
        if "Done" in os.listdir(self.batchPath):
            return
        now = time.time()
        fullData = self.importData(simulator)
        later = time.time()
        print('Total time for all data:{} seconds'.format(int(later-now)))
        count = 0
#         for pair in fullData:
#             np.save((os.path.join(self.batchPath, "Batch_" + str(count))), pair[0])
#             np.save((os.path.join(self.labelPath, "Label_" + str(count))), pair[1])
#             count += 1
        f = open(os.path.join(self.batchPath, "Done"), 'w')
        f.close()
        
    def readData(self):
        availableData = os.listdir(self.batchPath)
        if "Done" in availableData:
            availableData.remove("Done")
        indices = range(len(availableData))
        n_tasks = len(availableData)/self.threads
        print(n_tasks)
        q = Queue()
        if n_tasks == 0:
            return
        offSet = 0
        jobs = []
        feed = []
        for i in range(self.threads):
            p = Process(target=self.readFiles, args=(int(offSet), int(offSet+n_tasks), q))
            jobs.append(p)
            print("Starting")
            p.start() 
            offSet += n_tasks
        while not q.empty() or len(feed) != self.batch:
            feed.append(q.get())
        return feed
            
        
    def readFiles(self, fileRangeStart, fileRangeEnd):
        listL = []
        for i in range(fileRangeStart, fileRangeEnd):
            listL.append((str(i), np.load(os.path.join(self.batchPath, "Batch_" + str(i) + ".npy")), np.load(os.path.join(self.labelPath, "Label_"+ str(i) + ".npy"))))
        return listL

In [4]:
testSim = Market(['EUR','USD', 'TRY', 'JPY'], os.path.abspath('../Data_Processing/ProcessedData'))

In [5]:
data = DataManager(batch_size = 50, batch_count = 10000, batch_offset=0)
data.prepareData(testSim, reset=True)



Total time for all data:848 seconds


In [6]:
feedBackup = data.readFiles(0, 10000)

In [ ]:
print(np.array_equal(test2[0][2], test1[0][1]))


In [ ]:
def main():
    feed = feedBackup.copy()
    seeds = [3, 5, 7]
    tf.reset_default_graph()
    
    sess = tf.Session(config=tf.ConfigProto(log_device_placement=True))
    tf.set_random_seed(seeds[0])
    test1 = Agent(len(testSim.currencies), sess)
    sess.run(tf.global_variables_initializer())

    testData = feed[5]
    feed.pop(5)
    #print(testData)
    prices = []
    changes = []
    for data in feed:
        prices.append(data[1])
        changes.append(data[2])
        if np.isinf(data[1]).any() == True:
            print("FOUND in price")
        if np.isinf(data[2]).any() == True:
            print('FOUND in change')
    batch_size = 1000
    
#     testArr = np.zeros((4,1))
#     testArr[0] = -1
#     print(np.any(testArr < 0))
#     print(testSim.df['USDTRY'].iloc[323850:323949].values)
#     return
    
    priceMatrix = np.concatenate(prices)
    changeMatrix = np.concatenate(changes)
#     indexCheck = 3238
    #print(priceMatrix[indexCheck*batch_size:indexCheck*batch_size+batch_size][21])
    #print(changeMatrix[indexCheck*batch_size:indexCheck*batch_size+batch_size][21])
#     return
    indexSize = priceMatrix.shape[0]/batch_size
    try:
        for episode in range(50):
            print("Episode: {}".format(episode))
            index = list(range(int(indexSize)))
            batches = len(index)
            loss = 0
            count = len(feed)
            while len(index) != 0:
                for i in random.sample(index, 1):
                    with tf.device('/gpu:0'):
                        addToLoss = test1.train_step(priceMatrix[i*batch_size:i*batch_size + batch_size], changeMatrix[i*batch_size:i*batch_size + batch_size])
                        loss += addToLoss
                        if addToLoss < 0 or addToLoss == np.nan:
                            print('Loss at index {}: {}'.format(i,addToLoss))
                    index.remove(i)
            print(loss/batches)
    except KeyboardInterrupt:
        print("Ending")
        test1.test_model(testData[1], testData[2])
    finally:
        print("Ending")
        test1.test_model(testData[1], testData[2])
            
if __name__ == "__main__":
    main()

(?, 4, 1, 1000)
(?, 4, 1, 1)
Episode: 0
